In [69]:
import numpy as np
import pandas as pd
import networkx as nx



import os
import sys
import inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0, parentdir)

sys.path.insert(0, '..')

import Utils as Utils

In [71]:
# data_path = "RNA-Seq-Long-Read Sequencing of an Advanced Cancer Cohort Resolves Rearrangements, Unravels Haplotypes, and Reveals Methylation Landscapes/";
# file_name = "filtereddata/filtered_gene_exp_mat.csv";

# dataset = pd.read_csv(data_path+file_name,index_col=0);
# display(dataset);

# dataset.to_csv(data_path+file_name,index=None);

In [73]:
data_path = "RNA-Seq-Long-Read Sequencing of an Advanced Cancer Cohort Resolves Rearrangements, Unravels Haplotypes, and Reveals Methylation Landscapes/";
file_name = "expressionMatrix.tsv";

dataset = pd.read_csv(data_path+file_name,sep='\t',index_col=0).T;
display(dataset);

# dataset = Utils.string_to_numeric(dataset);
# display(dataset);

gene,AL513128.2,SNORA19,RN7SL384P,LINC00838,UCMA,AL731537.2,RPS4XP11,ECD,PSTK,FRMD4A,...,FAM243A,KRTAP6-3,VN1R7P,RPL37P4,KRTAP7-1,AF015720.1,RPSAP64,KRTAP19-4,AP001117.1,MIR6815
14891_T_F111558,0.02,0.0,0.0,0.00,0.00,0.0,0.03,14.09,1.11,4.97,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
17827_T_F94399,0.00,0.0,0.0,0.00,0.00,0.0,1.01,30.07,4.14,7.69,...,0.00,0.0,0.0,0.0,1.60,0.0,0.0,0.0,0.0,0.0
18153_T_D74656-F94400,0.00,0.0,0.0,0.00,0.00,0.0,0.26,24.91,4.37,40.40,...,0.12,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
18568_T_F112955,0.01,0.0,0.0,0.00,0.06,0.0,0.18,21.68,2.82,52.26,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
18843_T_F94483,0.00,0.0,0.0,0.00,0.00,0.0,1.76,17.23,5.33,0.63,...,0.00,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40120_T_F111531,0.00,0.0,0.0,0.00,0.00,0.0,0.00,5.96,1.47,18.51,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
40168_T_F94519,0.02,0.0,0.0,0.01,0.00,0.0,0.00,16.83,5.44,30.31,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
40489_T_F111532,0.01,0.0,0.0,0.00,0.00,0.0,0.00,14.83,2.63,19.31,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
42120_T_D74661-F48644,0.00,0.0,0.0,0.00,0.00,0.0,0.00,35.52,3.58,48.75,...,0.03,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [75]:
import scanpy as sc

# Create an AnnData object from the DataFrame
adata = sc.AnnData(X=dataset);
adata

AnnData object with n_obs × n_vars = 187 × 60650

In [76]:
adata.var["mito"] = adata.var_names.str.startswith("MT-");
sc.pp.filter_cells(adata, min_genes=500);
sc.pp.filter_genes(adata, min_cells=187);
adata

AnnData object with n_obs × n_vars = 187 × 14045
    obs: 'n_genes'
    var: 'mito', 'n_cells'

In [77]:
# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)


In [81]:
sc.pp.highly_variable_genes(adata, n_top_genes=180, inplace=True);
adata


AnnData object with n_obs × n_vars = 187 × 14045
    obs: 'n_genes'
    var: 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [82]:
adata.var.highly_variable
adata.X
print(adata)
print(adata.X)
display(adata.obs)
display(adata.var)

AnnData object with n_obs × n_vars = 187 × 14045
    obs: 'n_genes'
    var: 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
[[2.8935192  0.85173851 1.94825975 ... 1.71845448 4.64561439 3.97016753]
 [3.49166684 1.6833907  2.21296866 ... 1.33101021 4.42709862 3.83397744]
 [3.28116068 1.70333109 3.75020576 ... 1.62753423 2.90073971 2.51853678]
 ...
 [2.74289478 1.274561   2.99181575 ... 3.38257259 4.74135818 3.61808166]
 [3.5945372  1.51902941 3.90366269 ... 2.56945689 4.51350609 3.68604828]
 [2.98996728 1.22730965 2.33679666 ... 2.4806183  4.1330526  4.21415084]]


,n_genes
14891_T_F111558,28247
17827_T_F94399,30108
18153_T_D74656-F94400,33220
18568_T_F112955,28016
18843_T_F94483,29217
...,...
40120_T_F111531,26864
40168_T_F94519,28381
40489_T_F111532,29088
42120_T_D74661-F48644,26546


,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
gene,,,,,,
ECD,False,187,False,2.945731,1.093100,-1.245956
PSTK,False,187,False,1.531322,0.509675,-0.830412
FRMD4A,False,187,False,3.211888,4.284697,1.649690
PIPSL,False,187,False,0.301467,-1.445343,-1.322516
CC2D2B,False,187,False,0.251723,-1.075088,-0.863937
...,...,...,...,...,...,...
SSR4P1,False,187,False,0.801668,-0.141327,-0.845305
TEKT4P2,False,187,False,1.597886,2.058879,0.257718
FP565260.1,False,187,False,2.285622,1.675430,-0.374219


In [85]:
file_name = "filtereddata";

adata.write_csvs(data_path+file_name, skip_data=False, sep=',')

In [86]:
obs = pd.read_csv(data_path+file_name+"/obs.csv",index_col=0);
display(obs);
var = pd.read_csv(data_path+file_name+"/var.csv");
display(var);

dataset = pd.read_csv(data_path+file_name+"/X.csv",names=var['gene']);
dataset.index = obs.index;
display(dataset);

highly_variable_genes = var[var['highly_variable']]['gene'].values;
dataset = dataset[highly_variable_genes];
display(dataset);

,n_genes
14891_T_F111558,28247
17827_T_F94399,30108
18153_T_D74656-F94400,33220
18568_T_F112955,28016
18843_T_F94483,29217
...,...
40120_T_F111531,26864
40168_T_F94519,28381
40489_T_F111532,29088
42120_T_D74661-F48644,26546


,gene,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
0,ECD,False,187,False,2.945731,1.093100,-1.245956
1,PSTK,False,187,False,1.531322,0.509675,-0.830412
2,FRMD4A,False,187,False,3.211888,4.284697,1.649690
3,PIPSL,False,187,False,0.301467,-1.445343,-1.322516
4,CC2D2B,False,187,False,0.251723,-1.075088,-0.863937
...,...,...,...,...,...,...,...
14040,SSR4P1,False,187,False,0.801668,-0.141327,-0.845305
14041,TEKT4P2,False,187,False,1.597886,2.058879,0.257718
14042,FP565260.1,False,187,False,2.285622,1.675430,-0.374219
14043,IFNGR2,False,187,False,4.229868,2.864207,-0.707012


,ECD,PSTK,FRMD4A,PIPSL,CC2D2B,ADK,JCAD,AL360181.3,AGAP9,ITGB1,...,JAM2,COL6A1,VPS26C,SCAF4,MRPS6,SSR4P1,TEKT4P2,FP565260.1,IFNGR2,IL10RB
14891_T_F111558,2.893519,0.851739,1.948260,0.226511,0.226511,3.940256,1.831254,0.906999,1.439854,6.036033,...,1.456919,3.809700,3.223919,2.606643,4.147045,0.273643,0.701546,1.718454,4.645614,3.970168
17827_T_F94399,3.491667,1.683391,2.212969,0.838552,0.291806,3.858405,1.249437,0.283865,0.584539,5.237562,...,0.647438,3.936806,3.086111,2.595220,4.832503,0.431802,1.507515,1.331010,4.427099,3.833977
18153_T_D74656-F94400,3.281161,1.703331,3.750206,0.268797,0.394164,3.166670,2.590437,0.887427,1.631564,3.817501,...,3.704460,3.837413,3.294974,2.943881,3.918847,0.644277,0.660339,1.627534,2.900740,2.518537
18568_T_F112955,3.103091,1.326076,3.956301,0.128684,0.066411,3.187531,2.864402,0.653407,0.800282,4.684040,...,3.012306,7.279662,2.953186,2.800905,4.371978,0.880683,2.286275,1.540880,4.264054,3.864157
18843_T_F94483,2.919747,1.860173,0.495434,0.319605,0.274229,3.350647,0.297175,0.289584,0.390936,4.202319,...,3.996806,2.575061,3.350647,1.708198,5.219679,0.390936,2.935545,0.756006,3.915208,3.584986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40120_T_F111531,1.952541,0.912826,2.984614,0.068608,0.193138,2.066708,1.640850,1.291370,2.082003,5.248977,...,0.462743,4.442206,3.154067,2.816307,3.704941,0.281147,0.518634,2.248265,3.767347,3.658190
40168_T_F94519,2.860816,1.844590,3.423353,0.019390,0.226759,3.482227,1.194086,1.923424,2.698147,4.903890,...,1.240451,4.232233,3.679327,3.333585,4.003929,0.178808,0.721380,2.652291,4.826778,3.575534
40489_T_F111532,2.742895,1.274561,2.991816,0.009751,0.154079,2.542371,1.839191,1.191787,2.576477,5.159680,...,1.477196,4.675439,3.730620,3.122142,3.282929,1.285459,2.409837,3.382573,4.741358,3.618082
42120_T_D74661-F48644,3.594537,1.519029,3.903663,0.029459,0.696419,3.494406,0.972434,2.722698,1.206565,4.900253,...,1.345545,4.383267,3.815250,3.345203,4.233200,0.253875,0.430586,2.569457,4.513506,3.686048


,KCNMA1,ITIH5,PI15,TG,LPL,CCN3,CA2,PLAT,NR4A3,TMOD1,...,APCDD1,NFATC1,SEC11C,IGHG1,MEG3,SLC24A4,AL049828.1,AL135999.1,S100B,JAM2
14891_T_F111558,2.236974,0.820257,0.652337,0.216813,1.596213,1.085561,2.413113,2.781552,0.607199,0.607199,...,3.515975,1.727101,6.230087,10.546581,0.787753,0.403014,0.559927,0.664868,2.500250,1.456919
17827_T_F94399,3.731600,0.128969,0.156509,0.183311,1.055783,1.586395,3.952182,1.639157,0.517415,1.409020,...,7.421360,1.681422,5.336372,6.080286,0.613621,0.138234,0.275861,0.554585,1.018247,0.647438
18153_T_D74656-F94400,3.060263,1.857407,0.040296,0.069487,4.513324,1.611250,2.767957,2.221779,1.156657,3.893620,...,2.904125,0.717120,4.116626,0.161059,4.086175,0.143403,0.236866,0.394164,5.842557,3.704460
18568_T_F112955,0.562651,3.400752,0.904786,0.203443,5.742926,2.478034,1.446188,3.799549,0.717645,0.939885,...,3.468109,2.367625,2.081151,3.047381,6.051280,0.211415,0.075548,1.294323,3.429768,3.012306
18843_T_F94483,1.443758,0.030077,0.847288,1.889828,2.028870,2.545965,1.595073,0.942895,1.662929,1.127031,...,0.590039,0.297175,4.247133,0.312184,4.923746,0.030077,0.470315,0.418102,0.431413,3.996806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40120_T_F111531,0.700384,1.345647,0.068608,0.565822,0.481723,2.411014,5.371930,3.102080,0.250033,0.900534,...,2.486385,1.366559,3.963271,6.276362,3.534162,0.096630,0.512576,2.505615,0.403542,0.462743
40168_T_F94519,4.232375,2.283461,0.492450,0.404976,0.533475,1.324591,2.072011,2.214265,0.621371,0.657523,...,1.423645,3.037037,3.876886,6.419119,2.861376,0.730852,0.119793,1.268374,2.861936,1.240451
40489_T_F111532,2.353716,0.545299,0.084516,0.385501,0.637479,1.048023,1.304249,4.432766,0.111173,0.754657,...,2.935151,1.952456,4.096622,5.228770,2.827535,0.028973,0.119902,2.104408,0.250113,1.477196
42120_T_D74661-F48644,3.490464,1.701954,0.121825,0.749613,1.145072,0.624350,2.366212,0.397654,1.391199,0.404328,...,2.013275,4.342509,5.248508,3.730518,2.258703,0.725784,0.164994,2.431374,1.878096,1.345545


In [87]:
# dataset.corr()
dataset.to_csv(data_path+file_name+"/processed_data.csv",index=None);